In [ ]:
# math_challenge.py
import os
import sys
import time
import math
import random
os.environ.setdefault("PYGAME_HIDE_SUPPORT_PROMPT", "1")

try:
    import pygame
    pygame.init()
except ImportError:
    print("pygame is required. Please install it with: pip install pygame")
    sys.exit(1)
except pygame.error as e:
    print(f"Failed to initialize pygame: {e}")
    sys.exit(1)

# ---------------------- CONFIG ----------------------
W, H = 960, 600
FPS = 60
ROW_GAP = 136

SCROLL_SPEED = 1.6          # เริ่มช้า
MAX_SPEED = 5.5             # เพดานความเร็ว (เร่งตามเวลาแบบโค้งนุ่ม)

TITLE = "MATH CHALLENGE"
SAVE_FILE = "highscore.txt"

LEFT_W, RIGHT_W = 400, 200
ROW_H = 92
LEFT_BOX_X = 60
RIGHT_L_X = LEFT_BOX_X + LEFT_W + 30
RIGHT_R_X = RIGHT_L_X + RIGHT_W + 20

# สี
WHITE=(255,255,255); BLACK=(25,25,28); PINK=(230,60,140)
BG_TOP=(220,245,255); BG_BOTTOM=(210,230,245)
BLUE=(30,170,210); GREEN=(30,160,90); GREY=(120,120,130); RED=(225,60,60)

# ฟอนต์ (จะสร้างหลัง init)
FONT_MD = FONT_SM = FONT_XXL = None

# ---------------------- UTIL ----------------------
def draw_gradient(surf, c1, c2):
    try:
        h, w = surf.get_height(), surf.get_width()
        for y in range(h):
            t = y / h
            col = (
                int(c1[0]*(1-t) + c2[0]*t),
                int(c1[1]*(1-t) + c2[1]*t),
                int(c1[2]*(1-t) + c2[2]*t),
            )
            pygame.draw.line(surf, col, (0, y), (w, y))
    except (AttributeError, TypeError) as e:
        print(f"Error in draw_gradient: {e}")

def round_rect(surf, rect, color, r=16):
    try:
        pygame.draw.rect(surf, color, rect, border_radius=r)
    except pygame.error as e:
        print(f"Error in round_rect: {e}")

def shadow_rect(surf, rect, alpha=90, shadow=10, r=16):
    """เงานุ่มใต้กล่อง (รองรับ alpha เป็นชื่อพารามิเตอร์)"""
    try:
        s = pygame.Surface((rect.w + shadow*2, rect.h + shadow*2), pygame.SRCALPHA)
        pygame.draw.rect(s, (0,0,0,alpha), (shadow,shadow,rect.w,rect.h), border_radius=r)
        surf.blit(s, (rect.x - shadow, rect.y - shadow))
    except (pygame.error, AttributeError) as e:
        print(f"Error in shadow_rect: {e}")

def load_high():
    try:
        with open(SAVE_FILE, "r", encoding="utf-8") as f:
            v = f.read().strip()
            return int(v) if v else 0
    except (IOError, ValueError):
        return 0

def save_high(x):
    try:
        with open(SAVE_FILE, "w", encoding="utf-8") as f:
            f.write(str(x))
    except IOError as e:
        print(f"Could not save high score: {e}")

# ---------------------- PROBLEM GEN ----------------------
OPS = ["+","-","×","÷"]

def gen_add():
    a, b = random.randint(1,100), random.randint(1,100)
    return f"{a}+{b}", a+b

def gen_sub():
    a, b = random.randint(1,100), random.randint(1,100)
    return f"{a}-{b}", a-b   # อนุญาตติดลบ

def gen_div():
    # แบ่งลงตัวในช่วง 1..100
    while True:
        b = random.randint(1,100)
        qmax = 100 // b
        if qmax >= 1:
            q = random.randint(1, qmax)
            a = b * q
            if 1 <= a <= 100:
                return f"{a}÷{b}", q

def gen_mul():
    # ผลคูณที่อนุญาต: หลักสิบ (10..90) หรือ {100,150,200,250,300}
    valid_two   = [i*10 for i in range(1,10)]
    valid_three = [100,150,200,250,300]
    ans = random.choice(valid_two + valid_three)
    # หา a,b ในช่วง 1..100 ที่คูณได้ ans
    fac = [(a, ans//a) for a in range(1,101) if ans % a == 0 and 1 <= ans//a <= 100]
    a, b = random.choice(fac) if fac else (10, ans//10)
    return f"{a}×{b}", ans

def make_problem():
    op = random.choice(OPS)
    if op == "+":   text, ans = gen_add()
    elif op == "-": text, ans = gen_sub()
    elif op == "×": text, ans = gen_mul()
    else:           text, ans = gen_div()

    # ตัวลวงใกล้คำตอบ
    step = max(1, abs(ans)//12)
    deltas = [-3*step,-2*step,-step,step,2*step,3*step,5*step,-5*step]
    fake = ans + random.choice(deltas)
    if fake == ans: fake += step
    choices = [ans, fake]
    random.shuffle(choices)
    return text, ans, choices

# ---------------------- ROW ----------------------
class Row:
    def __init__(self, y, placeholder=False):
        self.y = y
        self.placeholder = placeholder
        if not placeholder:
            self.text, self.ans, self.choices = make_problem()
        else:
            self.text, self.ans, self.choices = "", None, []
        self.answered = False
        self.flash = 0
        self.ok = False
        self.sel = None

    def rects(self):
        L  = pygame.Rect(LEFT_BOX_X, self.y, LEFT_W, ROW_H)
        R1 = pygame.Rect(RIGHT_L_X, self.y, RIGHT_W, ROW_H)
        R2 = pygame.Rect(RIGHT_R_X, self.y, RIGHT_W, ROW_H)
        return L, R1, R2

    def update(self, dy):
        self.y += dy
        self.flash = max(0, self.flash - 1)

    def draw(self, surf):
        L, R1, R2 = self.rects()
        for r in [L, R1, R2]:
            shadow_rect(surf, r, alpha=110)
        round_rect(surf, L,  BLUE,  18)
        round_rect(surf, R1, GREEN, 18)
        round_rect(surf, R2, GREEN, 18)

        if not self.placeholder:
            t  = FONT_MD.render(self.text, True, WHITE)
            surf.blit(t,  t.get_rect(center=L.center))
            t1 = FONT_MD.render(str(self.choices[0]), True, WHITE)
            surf.blit(t1, t1.get_rect(center=R1.center))
            t2 = FONT_MD.render(str(self.choices[1]), True, WHITE)
            surf.blit(t2, t2.get_rect(center=R2.center))

        # เอฟเฟกต์ตอบถูก/ผิด (glow)
        if self.flash > 0 and self.sel:
            alpha = int(200 * (self.flash / 18))
            color = (60,255,120,alpha) if self.ok else (255,70,70,alpha)
            s = pygame.Surface((self.sel.w+28, self.sel.h+28), pygame.SRCALPHA)
            pygame.draw.ellipse(s, color, s.get_rect())
            surf.blit(s, (self.sel.x-14, self.sel.y-14))

    def click(self, pos):
        if self.placeholder or self.answered:
            return None
        _, R1, R2 = self.rects()
        if   R1.collidepoint(pos): pick = self.choices[0]; self.sel = R1
        elif R2.collidepoint(pos): pick = self.choices[1]; self.sel = R2
        else: return None
        self.answered = True
        self.ok = (pick == self.ans)
        self.flash = 18
        return self.ok

    def out_bottom(self):
        return self.y > H + 10

# ---------------------- SCREENS ----------------------
def draw_title(screen, best):
    draw_gradient(screen, BG_TOP, BG_BOTTOM)
    t = FONT_XXL.render(TITLE, True, PINK)
    screen.blit(t, t.get_rect(center=(W//2, 120)))

    play_rect = pygame.Rect(W//2-80, 260, 160, 160)
    shadow_rect(screen, play_rect, shadow=14, alpha=110)
    round_rect(screen, play_rect, PINK, 24)
    pygame.draw.polygon(screen, WHITE, [(W//2-18,290),(W//2-18,390),(W//2+34,340)])

    hs = FONT_SM.render(f"High Score: {best}", True, BLACK)
    screen.blit(hs, (24, 180))
    return play_rect

def draw_gameover(screen, score, best):
    draw_gradient(screen, (255,235,235), (245,220,220))
    t1 = FONT_XXL.render("TRY AGAIN", True, RED)
    t2 = FONT_SM.render(f"Score: {score}   High: {best}", True, BLACK)
    t3 = FONT_SM.render("Click anywhere to return", True, GREY)
    screen.blit(t1, t1.get_rect(center=(W//2,220)))
    screen.blit(t2, t2.get_rect(center=(W//2,300)))
    screen.blit(t3, t3.get_rect(center=(W//2,350)))

# ---------------------- CORE HELPERS ----------------------
def reset_game():
    rows = []
    y = -ROW_H
    while y < H - ROW_GAP:
        y += ROW_GAP
        rows.append(Row(y, False))

    # 4 แถวล่างสุด "ว่าง"
    for i in range(1, min(4, len(rows)) + 1):
        rows[-i].placeholder = True
        rows[-i].text = ""
        rows[-i].choices = []

    return rows, 0, SCROLL_SPEED, 0, 0, time.time()

# ---------------------- MAIN ----------------------
def main():
    global FONT_MD, FONT_SM, FONT_XXL

    try:
        screen = pygame.display.set_mode((W, H))
        pygame.display.set_caption(TITLE)
        clock = pygame.time.Clock()

        # Initialize fonts with fallback
        try:
            FONT_MD = pygame.font.Font(None, 48)
            FONT_SM = pygame.font.Font(None, 36)
            FONT_XXL = pygame.font.Font(None, 88)
        except pygame.error:
            print("Error loading fonts, using system default")
            FONT_MD = pygame.font.SysFont(None, 48)
            FONT_SM = pygame.font.SysFont(None, 36)
            FONT_XXL = pygame.font.SysFont(None, 88)

        high_score = load_high()

        state = "title"
        score = 0
        speed = SCROLL_SPEED
        rows = []
        shake = 0
        delay = 0
        start_time = 0

        running = True
        while running:
            try:
                dt = clock.tick(FPS) / 1000

                # INPUT
                for e in pygame.event.get():
                    if e.type == pygame.QUIT:
                        running = False
                    if e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE:
                        running = False
                    if e.type == pygame.MOUSEBUTTONDOWN and e.button == 1:
                        if state == "title":
                            p = draw_title(screen, high_score)
                            if p.collidepoint(e.pos):
                                rows, score, speed, shake, delay, start_time = reset_game()
                                state = "play"
                        elif state == "play":
                            for r in rows:
                                res = r.click(e.pos)
                                if res is None: 
                                    continue
                                if res:
                                    score += 1
                                else:
                                    shake = 14
                                    delay = 18
                                break
                        elif state == "gameover":
                            state = "title"

                # RENDER STATES
                if state == "title":
                    draw_title(screen, high_score)
                elif state == "play":
                    draw_gradient(screen, BG_TOP, BG_BOTTOM)
                    elapsed = time.time() - start_time
                    speed = SCROLL_SPEED + (MAX_SPEED - SCROLL_SPEED) * (1 - math.exp(-elapsed/60))
                    
                    for r in rows:
                        r.update(speed)
                        r.draw(screen)

                    rows = [r for r in rows if r.y <= H + ROW_GAP*6]
                    while rows and rows[-1].y >= ROW_GAP:
                        rows.insert(0, Row(rows[0].y - ROW_GAP))
                    if len(rows) > 18:
                        rows = rows[:18]

                    if delay == 0:
                        for r in rows:
                            if r.out_bottom() and not r.placeholder and not r.answered:
                                delay = 18
                                break

                    card = pygame.Rect(16, 14, 270, 64)
                    shadow_rect(screen, card, alpha=90)
                    round_rect(screen, card, WHITE, 16)
                    s1 = FONT_SM.render(f"Score: {score}", True, BLACK)
                    s2 = FONT_SM.render(f"Speed: {speed:.1f}", True, BLACK)
                    s3 = FONT_SM.render(f"FPS: {int(clock.get_fps())}", True, GREY)
                    screen.blit(s1, (card.x+14, card.y+8))
                    screen.blit(s2, (card.x+14, card.y+32))
                    screen.blit(s3, (card.x+160, card.y+8))

                    if delay > 0:
                        delay -= 1
                        if delay == 0:
                            high_score = max(high_score, score)
                            save_high(high_score)
                            state = "gameover"

                elif state == "gameover":
                    draw_gameover(screen, score, high_score)

                pygame.display.flip()

            except Exception as e:
                print(f"Error in game loop: {e}")
                running = False

    except pygame.error as e:
        print(f"Failed to initialize game: {e}")
    finally:
        save_high(high_score)
        pygame.quit()

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3973057985.py, line 89)